In [103]:
import numpy as np
import pandas as pd


In [104]:
x1 = np.random.randint(100,size=(20))
x2 = np.random.randint(100,size=(20))
x3 = np.random.randint(100,size=(20))


y1 = 2*x1 + 3*x2 + 4*x3

y2 = 7*x1 - 2*x2 + x3

In [105]:
XX1 =  {'col1': x1, 'col2': x2, 'col3': x3}
XX1 = pd.DataFrame(data= XX1)
XX1_numpy = XX1.to_numpy()

YY1 = {'col1': y1 }
YY1 = pd.DataFrame(data= YY1)
YY1_numpy = YY1.to_numpy()

YY2 = {'col1': y2 }
YY2 = pd.DataFrame(data= YY2)

In [106]:
YY2

,col1
0,607
1,393
2,530
3,-69
4,324
5,664
6,437
7,203
8,542
9,-111


In [107]:
input = XX1

In [108]:
weights_size = len(input.iloc[0])
solutions_per_problem = 10

pop_size = (solutions_per_problem, weights_size)
new_population = np.random.randint(low = 1, high= 5, size = pop_size)

In [109]:
new_population

array([[1, 2, 2],
       [4, 2, 2],
       [1, 2, 2],
       [3, 2, 2],
       [3, 1, 4],
       [2, 3, 4],
       [2, 2, 2],
       [3, 3, 1],
       [3, 4, 2],
       [4, 1, 1],
       [4, 3, 2],
       [3, 2, 2],
       [1, 1, 2],
       [1, 4, 3],
       [1, 3, 1],
       [2, 2, 4],
       [4, 1, 1],
       [4, 2, 1],
       [4, 2, 2],
       [1, 1, 3],
       [4, 4, 3],
       [4, 2, 3],
       [1, 4, 2],
       [1, 4, 3],
       [3, 2, 2],
       [4, 4, 2],
       [2, 1, 3],
       [4, 2, 4],
       [1, 2, 4],
       [3, 3, 4]])

In [110]:
# defining the starting constraints:

num_generations = 5

num_parents_mating = 4

sol_per_pop = 8



pop_size = (sol_per_pop,num_weights)


In [111]:
# fitness of each population:

fitness = np.empty(new_population.shape[0])

def cal_pop_fitness(target_dataframe, electricity_values, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    generated_electricity = np.sum(pop*electricity_values, axis=1);
    required_electricity =  target_dataframe['col1']
    error = required_electricity - generated_electricity
    error = error.abs()
    error = error.sum()
    fitness = -error
    
    return fitness


for i in range(len(new_population)):
    fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])

fitness

array([-4230., -1686., -4230., -2170., -1460.,     0., -3200., -2155.,
        -878., -3331., -1258., -2170., -5308., -1265., -4213., -1078.,
       -3331., -2377., -1686., -4247., -2077., -1317., -2224., -1265.,
       -2170., -1432., -3217., -1612., -2108., -1030.])

In [112]:
new_population.shape[1]

3

In [113]:
# Select the best parents for mating process:

number_of_parents = 5
pop = new_population

parents = np.empty((number_of_parents, new_population.shape[1]))

for instance in range(number_of_parents):
    max_index = np.where(fitness == np.max(fitness))
    max_index = max_index[0][0]
    parents[instance, :] = pop[instance, :]
    fitness[max_index] = -9999

parents

array([[1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 2., 2.],
       [3., 1., 4.]])

In [125]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)
 
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring






offspring_size = (pop_size[0]-parents.shape[0], weights_size)


crossover(parents, offspring_size)

array([[1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.]])

In [126]:
offspring

array([[1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.],
       [1., 2., 2.],
       [4., 2., 2.],
       [1., 2., 2.],
       [3., 1., 4.],
       [3., 2., 2.]])

In [127]:
new_population

array([[1, 2, 2],
       [4, 2, 2],
       [1, 2, 2],
       [3, 2, 2],
       [3, 1, 4],
       [2, 3, 4],
       [2, 2, 2],
       [3, 3, 1],
       [3, 4, 2],
       [4, 1, 1],
       [4, 3, 2],
       [3, 2, 2],
       [1, 1, 2],
       [1, 4, 3],
       [1, 3, 1],
       [2, 2, 4],
       [4, 1, 1],
       [4, 2, 1],
       [4, 2, 2],
       [1, 1, 3],
       [4, 4, 3],
       [4, 2, 3],
       [1, 4, 2],
       [1, 4, 3],
       [3, 2, 2],
       [4, 4, 2],
       [2, 1, 3],
       [4, 2, 4],
       [1, 2, 4],
       [3, 3, 4]])

In [62]:
num_generations = 5

num_parents_mating = 4
for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(equation_inputs, new_population)
    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
 
     # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))
 
     # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)
# Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

ValueError: Unable to coerce to DataFrame, shape must be (20, 3): given (10, 3)

In [63]:
def cal_pop_fitness(target_dataframe, electricity_values, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    generated_electricity = np.sum(pop*electricity_values, axis=1);
    required_electricity =  target_dataframe['col1']
    error = required_electricity - generated_electricity
    error = error.abs()
    error = error.sum()
    fitness = -error
    
    return fitness


def select_mating_pool(pop, fitness, num_parents_mating):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents_mating, pop.shape[1]))
    for parent_num in range(num_parents_mating):
        
        max_fitness_idx = np.where(fitness == np.max(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        parents[parent_num, :] = pop[max_fitness_idx, :]

        fitness[max_fitness_idx] = -9999

    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)
 
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
         # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
         # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
         # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
     
    return offspring


def mutation(offspring_crossover):

    # Mutation changes a single gene in each offspring randomly.

    for idx in range(offspring_crossover.shape[0]):

        # The random value to be added to the gene.

        random_value = np.random.uniform(-1.0, 1.0, 1)

        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value

    return offspring_crossover


In [68]:
#cal_pop_fitness(YY1, XX1, new_population)


for pop in new_population:
    print(cal_pop_fitness(YY1, XX1, pop))



-2586
-1163
-819
-1249
-819
-1858
-2677
-3217
-2586
-6104


In [14]:
## https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6 